In [0]:
import numpy as np
import pandas as pd
import keras
import csv

In [50]:
#!git clone https://github.com/thatbrguy/Dropbox-Uploader.git

Cloning into 'Dropbox-Uploader'...
remote: Enumerating objects: 951, done.
remote: Total 951 (delta 0), reused 0 (delta 0), pack-reused 951
Receiving objects: 100% (951/951), 318.69 KiB | 1.12 MiB/s, done.
Resolving deltas: 100% (506/506), done.


In [51]:
#cd Dropbox-Uploader

/content/Dropbox-Uploader/Dropbox-Uploader


In [0]:
#!chmod +x dropbox_uploader.sh


In [53]:
#!bash dropbox_uploader.sh


Dropbox Uploader v1.0
Andrea Fabrizi - andrea.fabrizi@gmail.com

Usage: dropbox_uploader.sh [PARAMETERS] COMMAND...

Commands:
	 upload   <LOCAL_FILE/DIR ...>  <REMOTE_FILE/DIR>
	 download <REMOTE_FILE/DIR> [LOCAL_FILE/DIR]
	 delete   <REMOTE_FILE/DIR>
	 move     <REMOTE_FILE/DIR> <REMOTE_FILE/DIR>
	 copy     <REMOTE_FILE/DIR> <REMOTE_FILE/DIR>
	 mkdir    <REMOTE_DIR>
	 list     [REMOTE_DIR]
	 monitor  [REMOTE_DIR] [TIMEOUT]
	 share    <REMOTE_FILE>
	 saveurl  <URL> <REMOTE_DIR>
	 search   <QUERY>
	 info
	 space
	 unlink

Optional parameters:
	-f <FILENAME> Load the configuration file from a specific file
	-s            Skip already existing files when download/upload. Default: Overwrite
	-d            Enable DEBUG mode
	-q            Quiet mode. Don't show messages
	-h            Show file sizes in human readable format
	-p            Show cURL progress meter
	-k            Doesn't check for SSL certificates (insecure)
	-x            Ignores/excludes directories or files from syncing.

In [0]:
#!echo "xO7ZN9X-ToAAAAAAAAAASBtUoCbS_NDfM-ukxdcFc0Nw1ST303tE-nTdXezTqT6V" > token.txt

In [55]:
#!bash dropbox_uploader.sh download Toucan


 > Downloading folder "/Toucan" to "/content/Dropbox-Uploader/Dropbox-Uploader/Toucan"... 
 > Creating local directory "/content/Dropbox-Uploader/Dropbox-Uploader/Toucan"... DONE
 > Downloading "/Toucan/test-inputs.txt" to "/content/Dropbox-Uploader/Dropbox-Uploader/Toucan/test-inputs.txt"... DONE
 > Downloading "/Toucan/testData.xlsx" to "/content/Dropbox-Uploader/Dropbox-Uploader/Toucan/testData.xlsx"... DONE
 > Downloading "/Toucan/testData.csv" to "/content/Dropbox-Uploader/Dropbox-Uploader/Toucan/testData.csv"... DONE


In [56]:
#TRAINING DATA
traindf = pd.read_csv('Toucan/testData.csv', delimiter = ',')
traindf = testdf.dropna(axis =1)
traindf.head()

,Sentence,QorA
0,Did his mother die of pneumonia,1
1,Is it derived from the Latin spoken by the rom...,1
2,Why did Bequerel win the Nobel Prize in Physics,1
3,Where is Jakarta located,1
4,"9 screens, but felt that was too much of a sa...",0


In [1]:
#!pip install spacy
#!python -m spacy download en_core_web_md
import spacy

In [0]:
nlp = spacy.load('en_core_web_md')

In [0]:
#TEST DATA AND TO NLP
in_txt = open("Toucan/test-inputs.txt").read().splitlines()
document = []
doc = []
for line in in_txt:
  doc.append(line)
  document.append(nlp(line))

In [63]:
#TRAINING DATA TO NLP
s_list = []
for index, row, q in traindf.itertuples():
    traindf['Sentence'][index] = nlp(traindf['Sentence'][index])
    s_list.append(traindf['Sentence'][index]) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
#GET TOKEN TAGS FROM TRAINING DATA
for t in s_list:
  for token in t:
    if token.tag_ == "WDT" or token.tag_ == "WP" or token.tag_ == "WP$" or token.tag_ == "WRB":
      wh_pos = token.tag_
      wh_word = token.text
      wh_bi_gram.append(token.text)
      wh_bi_gram.append(str(token.i + 1))
      if len(t)-1 != token.i:
        wh_nbor_pos = token.nbor()
      testdf['WHpos'][token.i] = wh_pos
      testdf['WHword'][token.i] = wh_word
      testdf['WHnbor'][token.i] = wh_nbor_pos
      testdf['WHbigram'][token.i] = wh_bi_gram
    if token.dep_ == "ROOT":
      root_token = token.tag_
      testdf['RootToken'][token.i] = root_token 
#train_df = get_token_tags(s_list)
question_train_df = testdf.copy()
question_train_df.drop(['Sentence', 'QorA'], axis =1, inplace = True)
#question_train_df.drop('QorA', axis = 1)
train_df = pd.get_dummies(question_train_df, columns = ['WHpos', 'WHword', 'WHnbor', 'RootToken'])
#train_df = pd.get_dummies(testdf, columns = ['WHpos', 'WHword', 'WHnbor', 'RootToken'])

In [2]:
#GET TOKEN TAGS FROM TEST DATA
predictdata = pd.DataFrame(columns = ['WHpos', 'WHword', 'WHnbor', 'RootToken'])
for t in document:
  for token in t:
    if token.tag_ == "WDT" or token.tag_ == "WP" or token.tag_ == "WP$" or token.tag_ == "WRB":
      wh_pos = token.tag_
      wh_word = token.text
      wh_bi_gram.append(token.text)
      wh_bi_gram.append(str(token.i + 1))
      if len(t)-1 != token.i:
       wh_nbor_pos = token.nbor()
      predictdata['WHpos'][token.i] = wh_pos
      predictdata['WHword'][token.i] = wh_word
      predictdata['WHnbor'][token.i] = wh_nbor_pos
      predictdata['WHbigram'][token.i] = wh_bi_gram
    if token.dep_ == "ROOT":
      root_token = token.tag_
      predictdata['RootToken'][token.i] = root_token
#predict_df = get_token_tags(document)
predict_df = pd.get_dummies(predictdata, columns = ['WHpos', 'WHword', 'WHnbor', 'RootToken'])

In [0]:
from sklearn.svm import LinearSVC

In [115]:
# CREATING DATA MATRIX
train = testdf['Sentence'].tolist()
predict = doc.copy() 
both_list = list(set(train + predict))
training_dict = {}
for val in both_list:
  if val not in testdf:
    training_dict[val] = [0 for i in range(len(testdf.index))]
  else:
    training_dict[val] = list(testdf[val])
XT_train = pandas.DataFrame(training_dict)
XT_train = csr_matrix(XT_train)

predicting_dict = {}
for col in both_list:
  if col not in predict:
    predicting_dict[val] = 0
  else:
    predicting_dict[val] = list(doc[val])  # KeyError

XT_predict = pandas.DataFrame(predciting_dict)
XT_predict = csr_matrix(XT_predict)

KeyError: ignored

In [0]:
#SVC
classdata = testdf['QorA']
lin = LinearSVC()
lin.fit(XT_train, classdata)
finalprediction = lin.predict(XT_predict)